# Functions and Methods and Multiple Dispatch
Manual: [Functions](https://docs.julialang.org/en/v1/manual/functions/), [Methods](https://docs.julialang.org/en/v1/manual/methods)

Functions are the primary building blocks in Julia. Every operation is done through functions (even math operations, like `+`).

In [ ]:
@edit 2+2 # (output is ugly in a notebook)

Functions return the result of the last evaluation, and this enables the short-hand notation

In [ ]:
# full form
function f(x)
    @show typeof(x)
    return x^2 # the keyword `return` can be omitted, but it's generally good to include for readability
end

# short form
g(x) = 2*x

@show g(2)

@show typeof(f(2))
@show typeof(f(2.0))
;

The parameter `x` above being untyped is unconstrained and takes `Any`. We get a different version from the compiled for each type passed in.

In [ ]:
@code_llvm g(2)
@code_llvm g(2.0)

In [ ]:
@code_native g(2)

We can constrain the type with an abstract type farther down the tree or with a concrete type and Julia will match with the closest match.

In [ ]:
g(x::Integer) = x + x
g(x::AbstractFloat) = x * x
g(x::Float32) = x
@show g(3)
@show g(3.0)
@show g(3.0f0)

h(x::Float32) = x
@show h(2)
;

This allows for a lot of flexibility in defining interfaces.

### Optional and Keyword Arguments

In [ ]:
# notice the semicolon
function keyword_optional(a, b, c=0; d=0, e) # put the optional one at the end in real life :)
    @show a, b, c, d, e
end

keyword_optional(1, 2, 3; d = 4, e = 5)

# keyword_optional(1, 2) # error: `e` not assigned

keyword_optional(1, 2, e=5) # semicolon at call is not required

# keyword_optional(1, 2, 3, 4, 5) # error because you didn't name `d` or `e`

### `Nothing` Functions
Similar to `void` (but with more usefulness), you can `return nothing` if you don't want a value returned from a function.

In [ ]:
function void1()
    return nothing
end
@show void1()
@show typeof(void1())

void2() = nothing
@show void2()
@show typeof(void2())

function void3()
    return
end
@show void3()
@show typeof(void3())
;

### Anonymous Functions
Similar to lambdas in Python or C++:

In [ ]:
# assigned to a variable
plusThree = (x, y) -> x + y + 3
@show plusThree(1,1)
sayHello = () -> println("Hello!")
@show sayHello()

# used in a function
a = [1, 2, 3]
b = map(x -> x^2, a) # for single argument, you can omit the ()
@show b
;

### Constructors and Functors

Manual: [Constructors](https://docs.julialang.org/en/v1/manual/constructors/),
[Functors](https://docs.julialang.org/en/v1/manual/methods/#Function-like-objects)

Julia types do not have member functions. The two (or 2.5) kinds of functions associated with types are constructors and functors.

In [ ]:
struct MyData
    x::Float64
    x2::Float64
    y::Float64
    z::Float64

    # inner constructor method (if you want to check for errors)
    function MyData(x::Float64, x2::Float64, y::Float64, z::Float64)
        if x < 0.0
            error("Can't use negative x value")
        end
        new(x,x2,y,z) # `new` available only to inner constructor methods
    end
end

# outer constructor method (useful for custom or simplified constructors)
function MyData(x::Float64, y::Float64)
    x2=x^2
    z = sin(x2+y)
    MyData(x, x2, y, z) # default constructor
end
    

println("Custom constructor: ", MyData(2.0, 3.0) )
println("Default constructor: ", MyData(2.0, 4.0, 3.0, 0.6569865987187891) )

In [ ]:
struct PrintFunc
    a::String
end

# Use the type instead of a function name
function (func::PrintFunc)(arg)
    println(func.a, " ", arg)
end

func = PrintFunc("hello")
# Then you can call an object as a function
func(2)
func("Philip")